## Actividad 1: Scrapear Frases v2.0

En esta actividad 1 del módulo 3 scrapearemos frases célebres de la página de Wikiquotes

Es importante que nos guiemos de la documentación: https://docs.scrapy.org/en/latest/ para saber qué está ocurriendo en el código y aprender nuevas formas de ejecutar el scraping

Con el código que vemos debajo podemos asegurarnos de instalar scrapy si es que no lo tenemos. Luego procedemos a importar el crawler que utilizaremos.

In [0]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
import re
import logging

Diseñamos las funciones de nuestro Pipeline.
* ```process_item``` es la función que se llama para procesar el objeto.
* ```remove_tags``` es una función personalizada

In [0]:
class ExtractLinePipeline(object):
    def process_item(self, item, spider):
        lines = dict(item)["quote"].splitlines()
        first_line = self.remove_tags(lines[0])
        #print(first_line)
        return {'quote': first_line}
    
    def remove_tags(self, text):
        html_tags = re.compile('<.*?>')
        return re.sub(html_tags, '', text)

Una vez tenemos definida nuestra Pipeline, procedemos a definir nuestro propio Spider. Atención a las ```custom_settings```.

In [0]:
class MarxQuotesSpider(scrapy.Spider):
    name = "MarxQuotesSpider"
    start_urls = [
        'https://es.wikiquote.org/wiki/Karl_Marx',
    ]
    
    custom_settings = {
        'LOG_LEVEL': logging.WARNING, # Con esto reducimos el número de mensajes que recibimos
        'ITEM_PIPELINES': {'__main__.ExtractLinePipeline': 1}, # Ejecuta nuestra Pipeline
        'FEED_FORMAT':'csv',                                 # Almacenamos la información de nuestra Pipeline
        'FEED_URI': 'marxquotes.csv'
    }
        
    def parse(self, response):
        for quote in response.css('div.mw-parser-output > ul > li'):
            #print(quote)
            yield {'quote': quote.extract()}

In [5]:
process = CrawlerProcess()

process.crawl(MarxQuotesSpider)
process.start()

2019-11-19 01:09:49 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: scrapybot)
2019-11-19 01:09:49 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.6.8 (default, Oct  7 2019, 12:59:55) - [GCC 8.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.14.137+-x86_64-with-Ubuntu-18.04-bionic
2019-11-19 01:09:49 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'csv', 'FEED_URI': 'marxquotes.csv', 'LOG_LEVEL': 30}
